In [2]:
from bs4 import BeautifulSoup
from collections import Counter, defaultdict
import json
from tqdm import tqdm

In [2]:
with open("data/href2dramapage.json", encoding="utf-8") as f:
    href2dramapage = json.load(f)

In [27]:
# href2details = defaultdict(dict)

for href, page in tqdm(href2dramapage.items()):
    if href not in href2details:
        
        soup = BeautifulSoup(page)
        details = soup.find_all("ul", {"class": "list m-b-0"})
        if details:
            
            for details_block in details:
                separate_details = details_block.find_all("li", {"class": "list-item p-a-0"})
                
                for separate_detail in separate_details:
                    
                    detail_prefix = separate_detail.find_all("b", {"class": "inline"})
                    assert len(detail_prefix) == 1
                    
                    detail_key = detail_prefix[0].get_text()
                    detail_value = separate_detail.get_text()
                    assert detail_value.startswith(detail_key)
                    
                    assert detail_key not in href2details[href]
                    href2details[href][detail_key] = detail_value

100%|████████████████████████████████████████████████████████████████████████████| 19462/19462 [30:23<00:00, 10.67it/s]


In [28]:
# %%time
# with open("data/href2details.json", "w", encoding="utf-8") as fw:
#     json.dump(href2details, fw, ensure_ascii=False)

CPU times: total: 922 ms
Wall time: 956 ms


In [30]:
len(href2dramapage)

19462

In [34]:
[href for href in href2dramapage if href not in href2details]

['/697659-big-mouth',
 '/697661-big-mouth',
 '/28640-see-you-again',
 '/744221-pretty-guardian-of-the-city',
 '/22041-magic-journey-to-africa',
 '/54603-run-bts-season-4',
 '/730725-reverse',
 '/716857-floor',
 '/32134-sanctuary',
 '/712575-shimobee',
 '/26061-faded',
 '/706905-the-remarried-empress',
 '/68071-beats-per-mizu',
 '/714027-the-hidden-fox',
 '/77971-off-gun-fun-night-season-2-special',
 '/23766-four-men',
 '/27100-dong-witch-and-me',
 '/685235-mentalist',
 '/20150-soulmates-the-series',
 '/22147-housewife-detective',
 '/17067-blue-in-my-heart',
 '/32734-hot-place',
 '/53895-wet-season',
 '/32769-younger',
 '/683333-triple-do-you-want',
 '/701285-last-twilight-in-phuket',
 '/682103-kirei-kamisama-to-machiawaseshita-onna',
 '/732917-cang-lan-jue',
 '/715703-cherry-maho-the-movie',
 '/15959-battle-of-nujiang',
 '/18836-angels-revenge',
 '/732821-new-untitled-sbs-reality-program',
 '/730863-we-are-the-champions',
 '/28343-saja',
 '/26952-2015-dream-concert',
 '/21912-the-happi

In [32]:
Counter([k for d in href2details.values() for k in d.keys()]).most_common()

[('Country:', 19393),
 ('Score:', 19393),
 ('Ranked:', 19393),
 ('Popularity:', 19393),
 ('Watchers:', 19393),
 ('Favorites:', 19393),
 ('Duration:', 18055),
 ('Episodes:', 13028),
 ('Aired:', 12258),
 ('Original Network:', 10738),
 ('Aired On:', 10189),
 ('Drama:', 9735),
 ('Movie:', 6365),
 ('Release Date:', 6365),
 ('TV Show:', 2101),
 ('Special:', 1192),
 ('Airs:', 770),
 ('Airs On:', 47)]

In [16]:
separate_details[0]

<li class="list-item p-a-0"><b class="inline">Score:</b> 8.2 <span class="hft">(scored by 12 users)</span></li>

In [17]:
details[1]

<ul class="list m-b-0"> <li class="list-item p-a-0"><b class="inline">Score:</b> 8.2 <span class="hft">(scored by 12 users)</span></li> <li class="list-item p-a-0"><b class="inline">Ranked:</b> #37553</li> <li class="list-item p-a-0"><b class="inline">Popularity:</b> #12666</li> <li class="list-item p-a-0"><b class="inline">Watchers:</b> 302</li> <li class="list-item p-a-0" style="display:none;"><b class="inline">Favorites:</b> 0</li> </ul>

In [3]:
with open("data/href2details.json", encoding="utf-8") as f:
    href2details = json.load(f)

In [4]:
import pandas as pd

df = pd.DataFrame({"details": href2details.values()})
df = pd.json_normalize(df["details"])
df["href"] = href2details.keys()

In [5]:
df.columns

Index(['Drama:', 'Country:', 'Episodes:', 'Aired:', 'Aired On:',
       'Original Network:', 'Duration:', 'Score:', 'Ranked:', 'Popularity:',
       'Watchers:', 'Favorites:', 'TV Show:', 'Movie:', 'Release Date:',
       'Special:', 'Airs:', 'Airs On:', 'href'],
      dtype='object')

In [6]:
for col in df.columns:
    if col.endswith(":"):
        df[col] = df[col].apply(
            lambda x: (x[len(col):].strip() if (type(x) == str) else x)
        )

In [7]:
df["Watchers:"] = df["Watchers:"].apply(
    lambda x: int(x.replace(",", ""))
)

In [8]:
cols = ["Country:"]
df.groupby(cols).agg({"Watchers:": ["median", "mean"]}).sort_values(("Watchers:", "median"))

Watchers:             
               median         mean
Country:                          
Philippines     114.5   556.150000
Hong Kong       202.0   390.447059
Japan           350.0  1232.555932
China           464.0  1940.104172
Thailand        611.0  2523.874016
Taiwan          621.0  2339.784257
South Korea     771.0  5113.669519

In [12]:
df.groupby("Country:").agg({"Original Network:": "nunique"}).sort_values("Original Network:")

,Original Network:
Country:,
Philippines,9
Hong Kong,16
Thailand,40
Taiwan,59
Japan,80
South Korea,137
China,171
